In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.master('local').appName('initDataFrames').enableHiveSupport().getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
orders = spark.read.csv('retail_db//orders.csv')

### Creation of Dataframes in PySpark - Method 1

In [ ]:
spark.createDataFrame([(1,)]).show()
#Creation of dataframes are enabled by list of tuples. Note that while mentioning tuples with one element then we need to specify comma at the end as above.

#### Using List

In [ ]:
l = [('Alice', 1)]
spark.createDataFrame(l).show()

In [ ]:
spark.createDataFrame(l, ['name', 'age']).show()

#### Using Dictionary (deprecated)

In [ ]:
d = [{'name': 'Alice', 'age': 1}]

In [ ]:
spark.createDataFrame(d).show()

#### Using rdd

In [ ]:
l = [('Alice', 1)]
rdd = sc.parallelize(l)
spark.createDataFrame(rdd).show()

In [ ]:
spark.createDataFrame(rdd, ['name', 'age']).show()

In [ ]:
spark.createDataFrame(rdd, "a: string, b: int").show()

#### Using Row

In [ ]:
Person = Row('name', 'age')
person = rdd.map(lambda x: Person(*x))
spark.createDataFrame(person).show()

### Creation of Pyspark Dataframes - Method 2
#### By loading the file as RDD and explicitly manipulating the fields and specifying datatypes

In [ ]:
ordersRDD = sc.textFile('retail_db//orders.csv')

In [ ]:
orders = ordersRDD.map(lambda x:(Row(x.split(',')[0],x.split(',')[1],x.split(',')[2],x.split(',')[3]))).toDF(). \
        toDF('order_id','order_date','order_customer_id','order_status')
orders.printSchema()

In [ ]:
orders.show()

### Creation of Pyspark Dataframes - Method 3
#### By loading the file as csv and mentioning the schema

In [ ]:
ordersSchema = StructType([
StructField('order_id',IntegerType(),False),
StructField('order_date',TimestampType(),False),
StructField('order_customer_id',IntegerType(),False),
StructField('order_status',StringType(),False)
])
orders = spark.read.csv('retail_db//orders.csv',header=True,schema=ordersSchema)
orders.printSchema()

In [ ]:
orders.show()

#### By loading the file as csv and modify datatypes

In [ ]:
orders = spark.read.csv('retail_db//orders.csv',header=True).toDF('order_id','order_date','order_customer_id','order_status')
orders.printSchema()

In [ ]:
orders.withColumn('order_id',orders.order_date.cast('int')).\
       withColumn('order_date',orders.order_date.cast('timestamp')).\
       withColumn('order_customer_id',orders.order_customer_id.cast('int')).\
       withColumn('order_status',orders.order_status.cast('string'))
orders.printSchema()

In [ ]:
orders.show()

#### By loading the file as csv and inferring schema automatically


In [ ]:
orders = spark.read.csv('retail_db//orders.csv',header=True,inferSchema=True,sep=',')
orders.printSchema()

In [ ]:
orders.show()